# M E T R O

In [61]:
import numpy as np
import pandas as pd
import seaborn as sns
import io
import ogr
import networkx as nx
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import random as ran
import ast 
import json
import lineas
import folium
import mplleaflet
import warnings

In [124]:
estaciones=nx.Graph()                      # Creamos una gráfica 
aristas = json.loads(lineas.jason)  

Función para medir la distancia entre dos puntos.

In [125]:
EARTH_R = 6372.8

def geocalc(lat0, lon0, lat1, lon1):
    """Return the distance (in km) between two points
    in geographical coordinates."""
    lat0 = np.radians(lat0)
    lon0 = np.radians(lon0)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    dlon = lon0 - lon1
    y = np.sqrt((np.cos(lat1) * np.sin(dlon)) ** 2 +
        (np.cos(lat0) * np.sin(lat1) - np.sin(lat0) *
         np.cos(lat1) * np.cos(dlon)) ** 2)
    x = np.sin(lat0) * np.sin(lat1) + \
        np.cos(lat0) * np.cos(lat1) * np.cos(dlon)
    c = np.arctan2(y, x)
    return EARTH_R * c

In [126]:
distancias = []
for key in aristas:
    i = 0
    ar = aristas[key]['coordinates']
    while i < len(ar)-1:
        x = (ar[i][1],ar[i][2])
        y = (ar[i+1][1],ar[i+1][2])
        estaciones.add_edge(x,y)
        distancias.append([x,y,geocalc(x[1], x[0], y[1], y[0])])
        i+=1

estaciones_list= list(estaciones)

In [131]:
for v,w in estaciones.edges:
   estaciones.edges[v,w]["weight"] = geocalc(v[1], v[0], w[1],w[0])
    

In [138]:
# Prueba para si las distancias son iguales.
estaciones.get_edge_data((-99.1555542, 19.4903266), (-99.1449058, 19.4896945))

{'weight': 1.1187284508765323}

In [132]:
#n1 = estaciones_list[ran.randint(0,163)]   #origen    nodos aleatorios
#n2 = estaciones_list[ran.randint(0,163)]

n1 = (-99.0976121386369,19.3223865035373) #lomas estrella
n2 = (-99.1627908,19.4236259) # insurgentes

In [151]:
path = nx.shortest_path(estaciones,n1,n2)
djs_path = nx.dijkstra_path(estaciones,n1,n2)
print("the shortest path is: " + str(path))

the shortest path is: [(-99.0976121386369, 19.3223865035373), (-99.1050339189064, 19.3276021253425), (-99.10888888888888, 19.3369444), (-99.1013199, 19.3561791), (-99.12166666666666, 19.3577778), (-99.1429317, 19.3619032), (-99.1415638, 19.369945), (-99.140260219574, 19.3795146127671), (-99.1390371, 19.3874695), (-99.1378087, 19.3952064), (-99.1368914, 19.4008785), (-99.1357434, 19.4084883), (-99.1441816, 19.4133303), (-99.1433448, 19.4216528), (-99.1422343, 19.4267827), (-99.149074, 19.42741), (-99.154653, 19.425867), (-99.1627908, 19.4236259)]


In [146]:
shortest_paths=[p for p in nx.all_shortest_paths(estaciones, n1,n2)]
shortest_paths[1]

[(-99.0976121386369, 19.3223865035373),
 (-99.1050339189064, 19.3276021253425),
 (-99.10888888888888, 19.3369444),
 (-99.1013199, 19.3561791),
 (-99.12166666666666, 19.3577778),
 (-99.1429317, 19.3619032),
 (-99.1415638, 19.369945),
 (-99.140260219574, 19.3795146127671),
 (-99.1390371, 19.3874695),
 (-99.1378087, 19.3952064),
 (-99.1368914, 19.4008785),
 (-99.1357434, 19.4084883),
 (-99.1441816, 19.4133303),
 (-99.1433448, 19.4216528),
 (-99.1422343, 19.4267827),
 (-99.149074, 19.42741),
 (-99.154653, 19.425867),
 (-99.1627908, 19.4236259)]

### Ploteamos la ruta más corta

In [153]:
lat =[]
lon = []
for i in range(len(path)):
    lon.append(path[i][0])
    lat.append(path[i][1])
    
plt.plot(lon,lat, 'r') 
plt.plot(lon,lat, 'rs')
mplleaflet.display()

### Ploteamos las rutas más cortas

In [150]:
lat =[]
lon = []
for i in range(len(shortest_paths[0])):
    lon.append(shortest_paths[0][i][0])
    lat.append(shortest_paths[0][i][1])
    
plt.plot(lon,lat, 'r') 
plt.plot(lon,lat, 'rs')
mplleaflet.display()

In [122]:
len(path)

18

In [123]:
mapa = folium.Map(location=[19.4284700,-99.1276600])  # inicilizamos con las coordenadas de la ciudad de mexico
for i in range(len(path)):
    folium.CircleMarker([path[i][1], path[i][0]], fill = True,
                       radius = 3, color = 'red', fill_opacity = 1).add_to(mapa)
    #folium.PolyLine(path, color="green", weight=2.5, opacity=1).add_to(my_map)
mapa
#mapa.save(('plot_data.html'))